In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.11.0+cpu


In [30]:
import torch
import pickle
import random
import numpy as np
import math
from torch_geometric.data import DenseDataLoader

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv, BatchNorm, GraphNorm, GCNConv, DenseGraphConv
from torch_geometric.nn import global_mean_pool, dense_diff_pool, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [5]:
with open(r'C:\\Users\\ivano\\Desktop\\DSMMaster\\CryptoThesis\\data\\address_subgraphs.pkl', "rb") as input_file:
    dataset = pickle.load(input_file)


In [23]:
print('====================')
print(f'Number of graphs: {len(dataset)}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of graphs: 102077


AttributeError: 'list' object has no attribute 'data'

In [7]:
classes = {dataset[i].y.index.map(str)[0]: dataset[i].y.item() for i in range(len(dataset))}
classes_1 = dict(filter(lambda i: i[1] == 1, classes.items()))
classes_0 = dict(filter(lambda i: i[1] == 0, classes.items()))

print(f'Number of high risk addresses: {len(classes_1.keys())}')
print(f'Number of safe addresses: {len(classes_0.keys())}')

# Balance the high risk and safe addresses (undersampling safe addresses)
random.seed(1993)
sample_safe = random.sample(list(classes_0.keys()), k = len(classes_1.keys()))

classes_0_trim = dict(filter(lambda i: i[0] in sample_safe, classes_0.items()))
classes_balanced = {**classes_1, **classes_0_trim}

Number of high risk addresses: 10169
Number of safe addresses: 91579


Need a way to get high risk graphs and then train against balanced number of licit graphs

In [10]:
classes_balanced_keys = list(classes_balanced.keys())
train_share = 0.8

train_keys = random.sample(classes_balanced_keys, k = round(train_share*len(classes_balanced_keys)))
test_keys = random.sample(classes_balanced_keys, k = len(classes_balanced_keys) - round(train_share*len(classes_balanced_keys)))

train_dataset = [graph for graph in dataset if graph.y.index[0] in train_keys] 
test_dataset = [graph for graph in dataset if graph.y.index[0] in test_keys]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing graphs: {len(test_dataset)}')

Number of training graphs: 16270
Number of testing graphs: 4068


In [11]:
for data in train_dataset:
  data.y = torch.tensor(data.y).type(torch.LongTensor)

for data in test_dataset:
  data.y = torch.tensor(data.y).type(torch.LongTensor) 

C:\Users\ivano\AppData\Local\Temp\ipykernel_14904\3021064243.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.y = torch.tensor(data.y).type(torch.LongTensor)


In [12]:
from torch_geometric.loader import DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
'''
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
'''

"\nfor step, data in enumerate(train_loader):\n    print(f'Step {step + 1}:')\n    print('=======')\n    print(f'Number of graphs in the current batch: {data.num_graphs}')\n    print(data)\n    print()\n"

In [13]:
# GAT

class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, heads, num_nodes):
        super(GAT, self).__init__()
        self.num_nodes = num_nodes
        #torch.manual_seed(12345)
        self.num_nodes = math.ceil(0.25 * self.num_nodes)
        self.gnn_pool_1 = GNN(18, hidden_channels, self.num_nodes)
        #self.gat_node_1 = GATConv(18, hidden_channels, edge_dim = 2, heads = heads)
        self.gnn_embed_1 = GNN(18, hidden_channels, hidden_channels)

        self.num_nodes = math.ceil(0.25 * self.num_nodes)
        self.gnn_pool_2 = GNN(hidden_channels, hidden_channels, self.num_nodes)
        #self.gat_node_2 = GATConv(heads * hidden_channels, hidden_channels, edge_dim = 2, heads = heads)
        self.gnn_embed_2 = GNN(hidden_channels, hidden_channels, hidden_channels, lin=False)

        #self.gat_node_3 = GATConv(heads * hidden_channels, out_channels, edge_dim = 2, heads = heads)
        self.gnn_embed_3 = GNN(hidden_channels, hidden_channels, hidden_channels, lin=False)

        self.lin = Linear(out_channels * heads, 2)
        
        self.graphnorm1 = GraphNorm(hidden_channels * heads)
        self.graphnorm2 = GraphNorm(hidden_channels * heads)
        self.graphnorm3 = GraphNorm(out_channels * heads)

    def forward(self, x, edge_index, edge_attr, batch):
        self.adj = to_dense_adj(edge_index)
        # 1. Obtain node embeddings 
        x = x.float()
        print(x.shape)
        s = self.gnn_pool_1(x, edge_index, None)
        #x = self.gat_node_1(x, edge_index, edge_attr = edge_attr)
        x = self.gnn_embed_1(x, edge_index, None)
        #x = self.graphnorm1(x)
        #x = F.leaky_relu(x)

        #x, self.adj, l1, e1 = dense_diff_pool(x, self.adj, s, None)
        #s = self.gnn_pool_2(x, edge_index, None)
        #x = self.gat_node_2(x, edge_index, edge_attr = edge_attr)
        #x = self.gnn_embed_2(x, edge_index, None)
        #x = self.graphnorm2(x)

        #x = F.dropout(x, p=0.5, training=self.training)
        #x = F.leaky_relu(x)
        x, self.adj, l1, e1 = dense_diff_pool(x, self.adj, s, None)

        #x = self.gat_node_3(x, edge_index, edge_attr = edge_attr)
        x = self.gnn_embed_3(x, self.adj, None)
        #x = self.graphnorm3(x)
        #x = F.dropout(x, p=0.5, training=self.training)
        #x = F.leaky_relu(x)
        
        x = x.mean(dim=1)
        # 2. Readout layer - turns node and edge embeddings into a graph embedding
        #x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin(x)
        x = F.softmax(x, dim = 1)
        
        return x



In [14]:
# DiffPool

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_nodes_batch, normalize=False, lin=True):
        super(GNN, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(GCNConv(in_channels, hidden_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(num_nodes_batch))
        
        self.convs.append(GCNConv(hidden_channels, hidden_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(num_nodes_batch))
        
        self.convs.append(GCNConv(hidden_channels, out_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(num_nodes_batch))


    def forward(self, x, adj, mask=None):
        #batch_size, num_nodes, _ = x.size()

        x = self.convs[0](x, adj)
        x = F.relu(x)

        x = self.convs[1](x, adj, mask)
        x = F.relu(x)

        x = self.convs[2](x, adj, mask)
        x = F.relu(x)

        #for step in range(len(self.convs)): # iterate through the three convolutional steps
        #  print(f'step {step}')
        #  x = self.bns[step](F.relu(self.convs[step](x, adj, mask)))

        return x

class DiffPool(torch.nn.Module):
    def __init__(self):
        super(DiffPool, self).__init__()
        
        self.max_nodes = 0
        for data in dataset:
          temp_max = data.num_nodes
          if temp_max>self.max_nodes:
            self.max_nodes = temp_max
        
        self.max_nodes = 50

        num_nodes_batch = self.max_nodes
        num_nodes = math.ceil(0.25 * self.max_nodes) 
        self.gnn1_pool = GNN(18, 64, num_nodes, num_nodes_batch) 
        self.gnn1_embed = GNN(18, 64, 64, num_nodes_batch)

        num_nodes_batch = num_nodes
        num_nodes = math.ceil(0.25 * num_nodes)

        self.gnn2_pool = GNN(64, 64, num_nodes, num_nodes_batch)
        self.gnn2_embed = GNN(64, 64, 64, num_nodes_batch, lin=False)

        num_nodes_batch = num_nodes
        self.gnn3_embed = GNN(64, 64, 64, num_nodes_batch, lin=False)

        self.lin1 = torch.nn.Linear(64, 64)
        self.lin2 = torch.nn.Linear(64, 32)

    def forward(self, x, edge_index, batch, mask=None):
        
        s = self.gnn1_pool(x, edge_index)
        x = self.gnn1_embed(x, edge_index)

        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)
        
        x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)
        #x_1 = s_0.t() @ z_0
        #adj_1 = s_0.t() @ adj_0 @ s_0
        
        s = self.gnn2_pool(x, edge_index)
        x = self.gnn2_embed(x, edge_index)

        x, adj, l2, e2 = dense_diff_pool(x, edge_index, s)

        x = self.gnn3_embed(x, edge_index)

        x = x.mean(dim=1)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2

In [19]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        num_nodes = math.ceil(0.5 * average_nodes)
        self.pool1 = Linear(hidden_channels, num_nodes)

        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        num_nodes = math.ceil(0.5 * num_nodes)
        self.pool2 = Linear(hidden_channels, num_nodes)

        self.conv3 = DenseGraphConv(hidden_channels, hidden_channels)

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()

        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)

        s = self.pool1(x)
        x, adj, mc1, o1 = dense_mincut_pool(x, adj, s, mask)

        x = self.conv2(x, adj).relu()
        s = self.pool2(x)

        x, adj, mc2, o2 = dense_mincut_pool(x, adj, s)

        x = self.conv3(x, adj)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mc1 + mc2, o1 + o2


In [36]:
average_nodes = 0

for data in dataset:
    average_nodes = average_nodes + int(data.x.size(0) / len(data))

average_nodes = average_nodes / len(dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GAT(hidden_channels = 128, out_channels = 32, heads = 10, num_nodes=150)
#model = DiffPool()

model = Net(in_channels = 18, out_channels = 32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#print(model)

In [40]:
import warnings
warnings.filterwarnings('ignore')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
         data = data.to(device)
         optimizer.zero_grad()  # Clear gradients.
         out, mc_loss, o_loss = model(data.x.type(torch.float), data.edge_index, data.batch) 
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.


def test(loader):
     model.eval()
        
     metrics = {"accuracy":0,
               "precision":0,
               "recall":0,
               "f1_score":0}

     correct = 0
     actual, predicted = list(), list()
     for data in loader:  # Iterate in batches over the training/test dataset.
         data = data.to(device)
         out, mc_loss, o_loss = model(data.x.type(torch.float), data.edge_index, data.batch)  
         pred = out.cpu().argmax(dim=1).tolist()  # Use the class with highest probability.
         true = data.y.cpu().tolist()
         correct += sum(x == y for x, y in zip(true, pred)) # Check against ground-truth labels.
         actual += true
         predicted += pred 

     metrics['accuracy'] = correct / len(loader.dataset)
     metrics['precision'], metrics['recall'], metrics['f1_score'], _ = precision_recall_fscore_support(actual, predicted, average = 'binary') # precision, recall, f1score
     
     return metrics  # Derive ratio of correct predictions.

epochs = 100
for epoch in range(epochs):
    train()
    train_metrics = test(train_loader)
    test_metrics = test(test_loader)

    #print(train_metrics)
    if epoch%10==0  or epoch==epochs:
      print(f'Epoch: {epoch:03d}') 
      print(f'Tr Acc: {train_metrics["accuracy"]:.3f}, Tr Rec: {train_metrics["recall"]:.3f}, Tr Pre: {train_metrics["precision"]:.3f}, Tr F1: {train_metrics["f1_score"]:.3f} '
            f'Te Acc: {test_metrics["accuracy"]:.3f}, Te Rec: {test_metrics["recall"]:.3f}, Te Pre: {test_metrics["precision"]:.3f}, Te F1: {test_metrics["f1_score"]:.3f}')

C:\Users\ivano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ivano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 000
Tr Acc: 0.503, Tr Rec: 0.000, Tr Pre: 0.000, Tr F1: 0.000Te Acc: 0.495, Te Rec: 0.000, Te Pre: 0.000, Te F1: 0.000


C:\Users\ivano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ivano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ivano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarni

KeyboardInterrupt: 